In [ ]:
import numpy as np

""" 
Note: simulation has to have realistic meanings
"""

In [ ]:
""" 
Simulation 1 scheme (Write down this mathematically):

code range: [1, 20]

Source domain:
    - Shape [10000, ]
    - Code 1,2,..., 8 are unique to source domain

Target domain:
    - Shape [10000, ]
    - Code 9,  are unique to target domain

Simulate features:
    - 1 layer: Sample parameters from Dirichlet distribution
    - 2(a) layer: Sample codes from the categorical distribution from the previous step
    - 2(b) layer: Sample codes from categorical distribution with equal probability
    - 3 layer: sample rate from Gaussian distirbution, and combine 2(a) and 2(b)

Simulate labels:
    - 1 layer: Sample n from categorical (n_features/4, n_features/3) (with equal probability)
    - 2(a) layer: If the n code with the highest underlying frequency exists, then 1
    - 2(b) layer: Sample codes from categorical distribution of [0, 1] with equal probability
    - 3 layer: sample rate from Gaussian distribution, and combine 2(a) and (b)




"""

Simulation 1 scheme 

Code range: 1,2,..., 20 

Source domain:\
    - Shape: [10000, ]\
    - Code 1,2,...,8 are more frequent

Target domain:\
    - Shape: [10000, ]\
    - Code 9, 10, 11, 12 are more frequent

Simulate features  
* 1. Set $\alpha = [\alpha_1, \alpha_2, ..., \alpha_{12}]$
    * 1(a). For males, $\alpha_1 =  ... = \alpha_8 = \frac{1}{10}$, $\alpha_9 = ...  = \alpha_12 = \frac{1}{20}$
    * 1(b). For females, $\alpha_1 =  ... = \alpha_8 = \frac{1}{20}, \alpha_9 = ...  = \alpha_12 = \frac{3}{20}$
* 2. Simulate $p_1,...,p_{12} \sim Dir(12, \alpha)$ 
* 3. Simulate $n \sim Cat(\frac{1}{5}, ..., \frac{1}{5})$ for $[10, 11, ..., 14]$
    * 3(a). Simulate $n_{male}$ for males
    * 3(b). Simulate $n_{female}$ for females
* 4. For each row (per patient, per sequence), simulate $a_1, ..., a_n \overset{\mathrm{iid}}{\sim} Cat(p_1,..., p_{12})$
* 5. For each row (per patient, per sequence), simulate $b_1, ..., b_n \overset{\mathrm{iid}}{\sim} Cat(\frac{1}{12},..., \frac{1}{12})$
* 6. Simulate $\epsilon \sim Gaussian(0, 1), \epsilon = min(|\epsilon|, 1)$, then, $c_1, ..., c_n = (1-\epsilon) [a_1, ..., a_n] + \epsilon[b_1, ..., b_n])$
* 7. Simulate $m \sim Cat(\frac{1}{5}, ..., \frac{1}{5})$ for $[6, 7, ..., 10]$
    * 7(a). Simulate $m_{male}$ for males
    * 7(b). Simulate $m_{female}$ for females
* 8. For each row (per patient, per sequence), simulate $d_1, ..., d_m \overset{\mathrm{iid}}{\sim} Cat(\frac{1}{m},..., \frac{1}{m})$
* 9. $x_1, ..., x_{n+m} = [c_1, ..., c_n, d_1, ..., d_m] $

Simulate labels
* 1. Simulate $k\sim \lfloor \frac{N}{4} \rfloor$
    * 1.(a) $N = 8$ for males
    * 1.(b) $N = 4$ for females
* 2. If the codes with the top $k$ highest $\alpha_i$ $(i \in [1,...,12])$ presents, then $e_x = 1$; otherwise, $e_x = 0$.
* 3. Simulate $f_x \sim Bernoulli(\frac{1}{2})$
* 4. Simulate $\delta \sim Gaussian(0, 1), \delta = min(|\delta|, 1)$, then $g_x = (1-\delta) e_x + \delta g_x$




